In [1]:
import tensorflow as tf
import torchfile

import numpy as np
import time, sys
import os

from model import build_model

In [2]:
# constants
width = 128
loss_lambda = 0.1

In [3]:
DATA_DIR = os.getcwd().rsplit('/', 1)[0] + '/data'
checkpoint_dir = os.getcwd().rsplit('/', 1)[0] + '/checkpoint'

In [4]:
def convert_joint_to_dict(joint):
    return dict(zip(joint.name, joint.position))

def restore_vars(saver, sess, checkpoint_dir, restore_flag=False):
    """ Restore saved net, global score and step, and epsilons OR
    create checkpoint directory for later storage. """
    sess.run(tf.initialize_all_variables())

    if not os.path.exists(checkpoint_dir):
        try:
            os.makedirs(checkpoint_dir)
        except OSError:
            pass

    if restore_flag:
        path = tf.train.latest_checkpoint(checkpoint_dir)
        if path is None:
            print ('No existing checkpoint found')
            return False
        else:
            print ('Restoring from %s' % path)
            saver.restore(sess, path)
            return True
    else:
        return False

In [5]:
# optimization
batch = 500
n_eval_interval = 5
n_train_step = 40
global_step = tf.Variable(0, trainable=False, name='global_step')
initial_learning_rate = 0.01
decay_steps = 64
decay_rate = 0.9
momentum = 0.5
learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate, staircase=True)

In [6]:
# model
grasp_class_prediction, logit, grasp_image_ph, keep_prob_ph = build_model(width)
grasp_class_ph =  tf.placeholder('int64', [None])

# loss
grasp_class_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=grasp_class_ph), name='grasp_class_loss')
combined_loss = grasp_class_loss

# evaluation
correct_prediction = tf.equal(tf.argmax(grasp_class_prediction, 1), grasp_class_ph)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# summary
tf.summary.scalar('learning_rate', learning_rate)
tf.summary.scalar('grasp_loss', grasp_class_loss)
tf.summary.scalar('loss', combined_loss)
tf.summary.scalar('accuracy', accuracy)
summary_op = tf.summary.merge_all()

Final shape [None, 16, 16, 32] flat_dim 8192


In [7]:
train_op = tf.train.AdamOptimizer(learning_rate).minimize(combined_loss, global_step=global_step)

In [8]:
def main():
    saver = tf.train.Saver(max_to_keep=5, keep_checkpoint_every_n_hours=1)
    with tf.Session() as sess:
        tf.set_random_seed(1234)
        np.random.seed(123)

        writer = tf.summary.FileWriter('tf-log/%d' % time.time(), sess.graph_def)

        restore_vars(saver, sess, checkpoint_dir)

        # load train data
        train_data = torchfile.load(DATA_DIR+'/train.t7')
        n = len(train_data[b'x'][0])
        print ('%d training samples' % n)
        print ('%d Loops/Epochs' % n_train_step)
        print ('%d Batch Size' % batch)
        for i in range(n_train_step):
            if i % n_eval_interval == 0:      
                ind = np.random.choice(n, int(batch/50), replace=False)
                rgb_image = train_data[b'x'][0][ind].transpose(0, 2, 3, 1)
                grasp_class = train_data[b'y'][ind] - 1
            else:
                ind = np.random.choice(n, batch, replace=False)
                rgb_image = train_data[b'x'][0][ind].transpose(0, 2, 3, 1)
                grasp_class = train_data[b'y'][ind] - 1

            if i % n_eval_interval == 0:
                val_feed = {
                    grasp_image_ph: rgb_image,
                    grasp_class_ph: grasp_class,
                    keep_prob_ph: 1.,
                }                

                print ('grasp loss', grasp_class_loss.eval(feed_dict=val_feed))
                print ('accuracy', accuracy.eval(feed_dict=val_feed))                
                writer.add_summary(sess.run(summary_op, feed_dict=val_feed), i)
                saver.save(sess, checkpoint_dir + '/model', global_step=i)

            # train
            train_feed = {
                grasp_image_ph: rgb_image,
                grasp_class_ph: grasp_class,
                keep_prob_ph: 0.8,
            }
            train_op.run(feed_dict=train_feed)

        # save the model
        saver.save(sess, checkpoint_dir + '/model', global_step=i)       


In [9]:
main()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
6415 training samples
40 Loops/Epochs
500 Batch Size
grasp loss 0.661511
accuracy 0.7
grasp loss 0.693147
accuracy 0.6
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 0.5
grasp loss 0.693147
accuracy 0.2
grasp loss 0.693147
accuracy 0.7
grasp loss 0.693147
accuracy 0.3
grasp loss 0.693147
accuracy 0.5


In [11]:
# optimization2
batch = 250
n_eval_interval = 5
n_train_step = 10**2
global_step = tf.Variable(0, trainable=False, name='global_step')
initial_learning_rate = 0.01
decay_steps = 64
decay_rate = 0.9
momentum = 0.5
learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate, staircase=True)
train_op = tf.train.AdamOptimizer(learning_rate).minimize(combined_loss, global_step=global_step)    
main()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
6415 training samples
100 Loops/Epochs
250 Batch Size
grasp loss 0.699326
accuracy 0.4
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 0.6
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 0.6
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 0.8
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 0.2
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 0.4
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.6
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.2


In [12]:
# optimization3
batch = 50
n_eval_interval = 5
n_train_step = 10**3
global_step = tf.Variable(0, trainable=False, name='global_step')
initial_learning_rate = 0.01
decay_steps = 64
decay_rate = 0.9
momentum = 0.5
learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate, staircase=True)
train_op = tf.train.AdamOptimizer(learning_rate).minimize(combined_loss, global_step=global_step)
main()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
6415 training samples
1000 Loops/Epochs
50 Batch Size
grasp loss 0.637325
accuracy 1.0
grasp loss 0.846277
accuracy 0.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.693147
accuracy 0.0
grasp loss 0.693147
accuracy 1.0
grasp loss 0.6